In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np 
import pandas as pd 
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import keras
from keras.preprocessing import image
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten,Dense,Dropout,BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, InceptionResNetV2, ResNet50, Xception
import cv2
from PIL import Image

from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten
from tensorflow.keras.optimizers import SGD


In [ ]:
path = '../input/plant-pathology-2021-fgvc8/'
train_dir = path + 'train_images/'
test_dir = path + 'test_images/'
#resized images
train_paths = '../input/resized-plant2021/img_sz_256'

In [ ]:
df = pd.read_csv('../input/plant-pathology-2021-fgvc8/train.csv')

In [ ]:
df2=pd.read_csv('../input/plant-pathology-2021-fgvc8/train.csv')

In [ ]:
df2['labels'].unique()

In [ ]:
df2['labels'] = df2['labels'].apply(lambda string: string.split(' '))
df2

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
s = list(df2['labels'])
mlb = MultiLabelBinarizer()
df2x = pd.DataFrame(mlb.fit_transform(s), columns=mlb.classes_, index=df2.index)
print(df2x.columns)

In [ ]:
df.head()

In [ ]:
df.labels.value_counts()


In [ ]:
df['labels'].unique()


In [ ]:
df['labels'] = df['labels'].astype(str)


In [ ]:
plt.figure(figsize=(8,5))
sns.countplot(data = df,y='labels')

In [ ]:
def plot_examples(label):
    fig, ax = plt.subplots(1, 5, figsize=(25, 5))
    ax = ax.ravel()
    for i in range(5):
        idx = df[df['labels']==label].index[i]
        image = cv2.imread(train_paths+df.loc[idx, 'image'])
        
        image =cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        ax[i].imshow(image)
        ax[i].set_title(label)
        ax[i].set_xticklabels([])
        ax[i].set_yticklabels([])

In [ ]:
#for labels in list(df['labels'].unique()):
 #   plot_examples(labels)

In [ ]:

#แปลงให้labelเป็นตัวเลข
count_dict = df.labels.value_counts()
label2id = {
    'scab': 0,
    'frog_eye_leaf_spot' : 1,
    'rust' : 2,
    'complex' : 3,
    'powdery_mildew' : 4,
}
NUM_CLASS = len(label2id)
id2label = dict([(value, key) for key, value in label2id.items()])
df["labels"] = df["labels"].map(lambda x : [i for i in x.split(" ") if i != "healthy"])
df["labels"] = df["labels"].map(lambda x : [label2id[i] for i in x])
df.head()

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                   validation_split = 0.2)
test_datagen = ImageDataGenerator(rescale = 1./255,
                                  validation_split = 0.2)

In [ ]:
train_generator = train_datagen.flow_from_dataframe(dataframe = df,
                                                   directory = train_paths,
                                                   target_size = (256,256),
                                                   x_col = 'image',
                                                   y_col = 'labels',
                                                   batch_size = 128,
                                                   color_mode = 'rgb',
                                                   class_mode = 'categorical',
                                                   subset = 'training')

test_generator = test_datagen.flow_from_dataframe(dataframe = df,
                                                 directory = train_paths,
                                                 target_size = (256,256),
                                                 x_col = 'image',
                                                 y_col = 'labels',
                                                 batch_size = 128,
                                                 color_mode = 'rgb',
                                                 class_mode = 'categorical',
                                                 subset = 'validation')

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), input_shape=(256,256,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(NUM_CLASS, activation='sigmoid')
])
ops = SGD(learning_rate=0.1)
model.compile(loss='binary_crossentropy', optimizer=ops,metrics=['accuracy'])
model.summary()

In [ ]:
#history = model.fit_generator(train_generator,validation_data,test_generator)
#history = model.fit_generator(train_generator,epochs = 15)
#history = model.fit_generator(train_generator,epochs = 1 ,validation_data = test_generator)



In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
checkpoint_path = "./muimodel.h5"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 monitor='val_loss',
                                                 save_best_only=True,
                                                 verbose=1)

In [ ]:
early_stopping = EarlyStopping(
                        monitor='val_loss',
                        min_delta=0.0,
                        patience=5,
                        verbose=1
                )

In [ ]:
BATCH_SIZE=128

In [ ]:
fit = model.fit_generator(train_generator,
                                  validation_data=test_generator,
                                  epochs=1,
                                  steps_per_epoch=test_generator.samples//BATCH_SIZE,
                                  validation_steps=test_generator.samples//BATCH_SIZE,
                                  callbacks=[cp_callback, early_stopping]
                                 )

In [ ]:
#plt.plot(history.history['loss'])
#plt.plot(history.history['val_loss']) 
#plt.title('Model Loss') 
#plt.ylabel('Loss') 
#plt.xlabel('Epochs') 
#plt.legend(['Train', 'Test'], loc='upper right') 
#plt.show()

# ------------------------------------------------------------